## Segmenting Neighborhoods in Toronto Notebook

In [2]:
import pandas as pd

In [4]:
! wget -O postalcodesT https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M


--2020-04-10 14:18:47--  https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
Resolving en.wikipedia.org... 2620:0:863:ed1a::1, 198.35.26.96
Connecting to en.wikipedia.org|2620:0:863:ed1a::1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 52229 (51K) [text/html]
Saving to: 'postalcodesT'

postalcodesT        100%[===================>]  51.00K  --.-KB/s    in 0.04s   

2020-04-10 14:18:47 (1.14 MB/s) - 'postalcodesT' saved [52229/52229]



In [16]:
# read wget file. I got lucky as I saw it was a list and put[0] after and found that it read it all. 
df = pd.read_html('postalcodesT')[0]
df

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
...,...,...,...
175,M5Z,Not assigned,NaN
176,M6Z,Not assigned,NaN
177,M7Z,Not assigned,NaN
178,M8Z,Etobicoke,Mimico NW / The Queensway West / South of Bloo...


In [20]:
# find all the Boroughs who do not have a name and drop them
not_assigned_index = df.index[df['Borough'] == 'Not assigned']
df.drop(index=not_assigned_index, inplace=True)  

In [30]:
# There are no Neighborhoods who have Null values
df[df['Neighborhood'].isnull()]

,Postal code,Borough,Neighborhood


In [33]:
#Reset Dataframe Index for clarity's sake
df = df.reset_index(drop=True)

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing CentrE
101,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...


In [34]:
df.shape

(103, 3)